## Init

In [ ]:
import os

path = os.getcwd()
# find the string 'project' in the path, return index
index_project = path.find('project')
# slice the path from the index of 'project' to the end
project_path = path[:index_project+7]
# set the working directory
os.chdir(project_path+'/src')
print(f'Project path set to: {os.getcwd()}')

In [ ]:
from dotenv import dotenv_values
config = dotenv_values(".env")
print(config["DATA_PATH"])

In [ ]:
from models.ModelBuilder import ModelBuilder
from models.Reaction import Reaction
from models.ReactionArchtype import ReactionArchtype
from models.ArchtypeCollections import *

# import scikit-learn
from sklearn.linear_model import LinearRegression
# tree models and support vector machines
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import pearson correlation
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from copy import deepcopy

## Notebook Parameters

In [ ]:
import os 

### parameters 

notebook_name = 'exp19_model_topology' # name of the notebook

## Generation of ground truth model 

model_name = 'v4_drug_model' # name of the model
o_random_seed = 53252
# p_overall_seed = 46 # different seed for parameter generation
no_observable_species = 13
no_feedback_regulations = 7
specie_value_range = (1000, 5000)
param_range = (0.8, 1.2)
param_multiplier_range = (0.99, 1.01)


## Simulation parameters 

simulation_time = 1000 
simulation_step = 100

## Feature data generation 

feature_generation_method = 'lhs'
feature_generation_extra_params = {'min': 0.1, 'max': 10}
feature_generation_size = 1000 
feature_generation_seed = 50 # if -1 then 'o_random_seed' is used
if feature_generation_seed == -1:
    feature_generation_seed = o_random_seed
    
## Data engineering parameters

# Suboptimal Model Generation 

'''
Options: 
- 'feedback_prune': removes feedback regulations from the model 
- 'random parameter': randomizes a x% of parameter values of the model
'''

''' 
Options: 
- 'last_time_point' : only the last time point of the phosphorylated species is used
- 'dynamic_feature': computes the characteristic 'ten' dynamic feature for each specie data 
'''

## General parameters
parallelise = True
save_figures = True 
experiment_id = notebook_name + '_' + str(o_random_seed) + '_' + str(feature_generation_seed)
experiment_folder = config['DATA_PATH'] + '/' + experiment_id + '/'
if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)
    
print(experiment_folder)

## Analysis

### Virtual Cell Creation

In [ ]:
# create a drug enabled model 
from models.Utils import *
from models.DrugModelSpecification import DrugModelSpecification, Drug

model_drug_spec = DrugModelSpecification()
model_drug_spec.generate_specifications(o_random_seed, no_observable_species, no_feedback_regulations, verbose=0)
drug_0 = Drug('D0', 500, 500)
rng = np.random.default_rng(o_random_seed)
# add random 'up' and 'down' regulations to the drug
regulation_dir = []
for i, s in enumerate(model_drug_spec.A_species):
    reg_type = str(rng.choice(['up', 'down']))
    regulation_dir.append(reg_type)
    drug_0.add_regulation(s, reg_type)
model_drug_spec.add_drug(drug_0)
print(model_drug_spec)
print(f'Feedback: {model_drug_spec.get_feedback_regulations()}')

In [ ]:
p_random_seeds = []
feature_size = 1000 
rng = np.random.default_rng(o_random_seed)
# generate `feature_size` random seeds for different parameter sets using numpy, ensure that the seeds are unique
p_random_seeds = rng.choice(range(1000000), feature_size, replace=False).tolist()

In [ ]:
G0_d = model_drug_spec.generate_network('drug_model_524', 
                                        specie_value_range, 
                                        param_range, 
                                        param_multiplier_range,  
                                        verbose=0,
                                        random_seed=p_random_seeds[0])
base_parameters = G0_d.get_parameters()
base_initial_conditions = G0_d.get_state_variables()

print(G0_d.get_antimony_model())

In [ ]:
parameter_sets = []
for p in p_random_seeds: 
    model_build = model_drug_spec.generate_network(f'param_seed_{p}', 
                                            specie_value_range, param_range, param_multiplier_range, random_seed=p, verbose=0)
    parameter_sets.append(model_build.get_parameters())

In [ ]:
# test simulation 

from models.Solver.RoadrunnerSolver import RoadrunnerSolver

solver = RoadrunnerSolver()
solver.compile(G0_d.get_sbml_model())

original_result = solver.simulate(0, 1000, 100)
original_result

### Perform parameter sensitivity analysis on original model

In [ ]:
from sklearn.metrics import mean_squared_error

original_cp = original_result['Cp']
sensitivity_summary = []
parameters_list = G0_d.get_parameters()

for key, value in parameters_list.items():
    factor_range = np.arange(0.9, 1.11, 0.02)
    for factor in factor_range:
        new_value = value * factor
        solver.set_parameter_values({key: new_value})
        result = solver.simulate(0, 1000, 100)
        perturbed_cp = result['Cp']
        
        # Calculate RMSE or other metric
        euclid_norm = np.linalg.norm(original_cp - perturbed_cp)
        
        # Store result
        sensitivity_summary.append({
            'parameter': key,
            'factor': factor,
            'rmse': euclid_norm,
        })

# Convert to DataFrame
sensitivity_df = pd.DataFrame(sensitivity_summary)


In [ ]:
# Compute mean Euclidean norm per parameter
rank_df = sensitivity_df.groupby('parameter')['rmse'].mean().reset_index()
rank_df = rank_df.sort_values(by='rmse', ascending=False)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Compute mean Euclidean norm and sort
rank_df = (
    sensitivity_df.groupby('parameter')['rmse']
    .mean()
    .reset_index()
    .sort_values(by='rmse', ascending=False)
)

# Select top 10
top10_df = rank_df.head(10)

# Plot
plt.figure(figsize=(8, 5))
sns.set(style="whitegrid", context="talk")

sns.barplot(
    data=top10_df,
    x='rmse',
    y='parameter',
    hue='parameter',
    palette='Blues_r',
)

plt.title("Top 10 Most Sensitive Parameters (Mean Euclidean Norm of Cp)")
plt.xlabel("Mean Euclidean Norm")
plt.ylabel("Parameter")
plt.tight_layout()
plt.show()


In [ ]:
import re
import pandas as pd
from collections import defaultdict

def infer_phosphorylation_map(antimony_str):
    """
    Infers base species to phosphorylated form based on 'Xp' species in the model.
    """
    # Match all species names (A0, A0p, etc.)
    species = set(re.findall(r'\b[A-Za-z]\w*\b', antimony_str))
    
    mapping = {}
    for sp in species:
        if sp.endswith('p'):
            base = sp[:-1]
            if base in species:
                mapping[base] = sp
    return mapping


def map_regulations_to_parameters(antimony_str, regulations, regulation_types=None):
    """
    Map each regulation to its controlling parameters based on rate laws in the Antimony model.
    
    Parameters:
        antimony_str (str): Antimony model string.
        regulations (list of tuples): Each tuple is (regulator, target).
        regulation_types (list of str, optional): 'up' or 'down' for each regulation.
    
    Returns:
        pd.DataFrame: DataFrame with columns: ['regulator', 'target', 'type', 'parameters']
    """
    # Map regulators to their phosphorylated forms
    phosphorylated_form = infer_phosphorylation_map(antimony_str)

    # Parse reactions
    pattern = re.compile(r'(J\d+):\s*(\w+)\s*->\s*(\w+);\s*(.*)')
    reactions = pattern.findall(antimony_str)
    
    # Map target species to associated rate expressions
    target_to_rates = defaultdict(list)
    for rxn_id, reactant, product, rate_expr in reactions:
        target_to_rates[reactant].append((rxn_id, rate_expr))
    
    results = []
    for i, (reg, tgt) in enumerate(regulations):
        reg_form = phosphorylated_form.get(reg, reg)
        r_type = regulation_types[i] if regulation_types else "unknown"
        matched_params = set()

        # Search all rate expressions that involve the target
        for rxn_id, rate_expr in target_to_rates.get(tgt, []):
            # Match: reg_form * param or param * reg_form
            matches = re.findall(
                rf'\b{reg_form}\b\s*\*\s*(\w+)|(\w+)\s*\*\s*\b{reg_form}\b',
                rate_expr
            )
            found = [m[0] or m[1] for m in matches if m[0] or m[1]]
            matched_params.update(found)

        results.append({
            'regulator': reg,
            'target': tgt,
            'type': r_type,
            'parameters': sorted(matched_params) if matched_params else None
        })

    return pd.DataFrame(results)


In [ ]:
import re
import pandas as pd

def extract_phosphorylation_as_regulations(antimony_str):
    """
    Extract phosphorylation reactions as regulator-target pairs with Vmax/Km.
    
    Returns:
        pd.DataFrame with columns: ['regulator', 'target', 'type', 'parameters']
    """
    rxn_pattern = re.compile(r'(J\d+):\s*(\w+)\s*->\s*(\w+);\s*(.*)')
    reactions = rxn_pattern.findall(antimony_str)

    records = []
    for rxn_id, reactant, product, rate_expr in reactions:
        # Heuristic: phosphorylation if product ends with 'p' and matches reactant + 'p'
        if product.endswith('p') and reactant == product[:-1]:
            vmax_match = re.search(r'([A-Za-z0-9_]*Vmax)', rate_expr)
            km_match = re.search(r'([A-Za-z0-9_]*Km)', rate_expr)

            vmax = vmax_match.group(1) if vmax_match else None
            km = km_match.group(1) if km_match else None
            params = list(filter(None, [vmax, km]))

            records.append({
                'regulator': reactant,
                'target': product,
                'type': 'phosphorylation',
                'parameters': params if params else None
            })

    return pd.DataFrame(records)


In [ ]:
regulations = model_drug_spec.regulations
regulation_types = model_drug_spec.regulation_types

df = map_regulations_to_parameters(
    G0_d.get_antimony_model(),
    regulations,
    regulation_types
)

In [ ]:
df

In [ ]:
phospho_reg_df = extract_phosphorylation_as_regulations(G0_d.get_antimony_model())


In [ ]:
phospho_reg_df

In [ ]:
# join two df into a new df called 'regulation_df' 
regulation_df = pd.concat([df, phospho_reg_df], ignore_index=True)

In [ ]:
def rank_regulations_by_sensitivity(sensitivity_df, regulation_df):
    """
    Join sensitivity scores to regulations based on parameter names,
    and rank regulations by max sensitivity of any associated parameter.
    
    Returns:
        DataFrame with ['regulator', 'target', 'type', 'parameters', 'mean_rmse', 'max_rmse']
    """
    # Explode regulation parameters
    exploded = regulation_df.explode('parameters').dropna(subset=['parameters'])
    exploded = exploded.rename(columns={'parameters': 'parameter'})

    # Join with sensitivity values
    merged = pd.merge(exploded, sensitivity_df, on='parameter', how='left')

    # Aggregate sensitivity per regulation
    ranked = (
        merged.groupby(['regulator', 'target', 'type'])
        .agg(
            parameters=('parameter', lambda x: sorted(set(x))),
            mean_rmse=('rmse', 'mean'),
            max_rmse=('rmse', 'max')
        )
        .reset_index()
        .sort_values(by='max_rmse', ascending=False)
    )

    return ranked


In [ ]:
model_str = G0_d.get_antimony_model() 

# Combine both regulation + phosphorylation regulation tables
all_regulations_df = pd.concat([
    map_regulations_to_parameters(model_str, regulations, regulation_types),
    extract_phosphorylation_as_regulations(model_str)
], ignore_index=True)

# Rank regulations
ranked_regs = rank_regulations_by_sensitivity(sensitivity_df, all_regulations_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_top_sensitive_regulations(ranked_df, top_n=10):
    """
    Plot bar chart of top sensitive regulations by max_rmse.
    """
    # Create a readable label for the regulation
    ranked_df['label'] = ranked_df['regulator'] + " → " + ranked_df['target'] + " (" + ranked_df['type'] + ")"

    # Select top N
    top_df = ranked_df.nlargest(top_n, 'mean_rmse')

    # Plot
    plt.figure(figsize=(10, 6))
    sns.set(style="whitegrid", context="talk")

    sns.barplot(
        data=top_df,
        x='mean_rmse',
        y='label',
        palette='Reds_r',
        hue='label',
    )

    plt.xlabel("Mean Euclidean Norm (Cp deviation)")
    plt.ylabel("Regulation")
    plt.title(f"Top {top_n} Most Sensitive Regulations")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_top_sensitive_regulations(ranked_regs, top_n=10)


### Generate synthetic 'omics-like' data

In [ ]:
from models.SyntheticGen import generate_feature_data, generate_target_data, generate_feature_data_v2, generate_target_data_diff_build

feature_data = generate_feature_data_v2(model_drug_spec, base_initial_conditions, feature_generation_method, feature_generation_extra_params, 1000, feature_generation_seed)
target_data, _ = generate_target_data_diff_build(model_drug_spec, solver, 
                                                 feature_data, parameter_sets, 
                                                 {'start': 0, 'end': 1000, 'points': 100}, 
                                                 n_cores=1, verbose=True)


In [ ]:
pd.to_pickle(feature_data, config["DATA_PATH"] + "/presentations/feature_data_v1.pkl")
pd.to_pickle(target_data, config["DATA_PATH"] + "/presentations/target_data_v1.pkl")

In [ ]:
# plot the association between the features and the target data in a bar chart based on the correlation values 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# create a dataframe with the feature data and target data
feature_data_df = pd.DataFrame(feature_data)
target_data_df = pd.DataFrame(target_data)
# add the target data to the feature data
feature_data_df['target'] = target_data_df['Cp']

# calculate the correlation between the features and the target data
correlation = feature_data_df.corr()['target'].sort_values(ascending=False)
# create a dataframe with the correlation values
correlation_df = pd.DataFrame(correlation)
correlation_df = correlation_df.reset_index()
correlation_df.columns = ['feature', 'correlation']
# do not include the target data in the correlation dataframe
correlation_df = correlation_df[correlation_df['feature'] != 'target']
abs_correlation_df = correlation_df.copy()
# take the absolute value of the correlation
abs_correlation_df['correlation'] = abs(abs_correlation_df['correlation'])
# sort the dataframe by the absolute value of the correlation
abs_correlation_df = abs_correlation_df.sort_values(by='correlation', ascending=False)
abs_correlation_df_50 = abs_correlation_df.head(50)

sns.set_context("talk", rc={"font": "Arial"})
sns.set_style('whitegrid')

# plot the correlation values
plt.figure(figsize=(8, 4))
sns.barplot(x='feature', y='correlation', data=abs_correlation_df_50)
plt.title('Correlation between features and target data')
plt.xlabel('Feature')
plt.ylabel('Abs Pearson Correlation')
plt.xticks(rotation=90, fontsize=14)
plt.grid()
plt.show()

In [ ]:
# make a histogram of the correlation values
plt.figure(figsize=(20, 10))
sns.histplot(abs_correlation_df['correlation'], bins=50, kde=True)
plt.title('Histogram of Absolute correlation values')
plt.xlabel('Abs Pearson Correlation')
plt.ylabel('Frequency')
plt.xlim(0,1)
plt.show()

In [ ]:
# make a histogram of the correlation values
sns.set_context("talk", rc={"font": "Arial"})
sns.set_style('whitegrid')
plt.figure(figsize=(8, 4), dpi=300)
sns.histplot(correlation_df['correlation'], bins=50, kde=True)
plt.title('Histogram of correlation values')
plt.xlabel('Abs Pearson Correlation')
plt.ylabel('Frequency')
plt.xlim(-0.5,0.5)
plt.show()

In [ ]:
# extract the mean and standard deviation of the correlation values
mean = np.mean(correlation_df['correlation'])
std = np.std(correlation_df['correlation'])
max_val = np.max(correlation_df['correlation'])
min_val = np.min(correlation_df['correlation'])
outliers = correlation_df[(correlation_df['correlation'] > mean + 3*std) | (correlation_df['correlation'] < mean - 3*std)]

print(f'Mean: {mean:.4f}')
print(f'Standard Deviation: {std:.4f}')
print(f'Max: {max_val:.4f}')
print(f'Min: {min_val:.4f}')
print(f'Outliers as a ratio of all features: {outliers.shape[0] / correlation_df.shape[0]:.4f}')

In [ ]:
# plot association between the features and the target data 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# for each feature, plot the association with the target data, which is only a series 
# plot all features in separate subplots
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1.5) 

n_features = len(feature_data.columns)
# plot only the top 10 features based on the absolute correlation values
top_features = abs_correlation_df['feature'].values[:10]
# create a new dataframe with only the top features
top_feature_data = feature_data[top_features]
print(f'{len(top_feature_data.columns)} features selected based on correlation with target data')
# make a subplot based on the size of feature_data, have five columns and as many rows as needed
top_n_features = len(top_feature_data.columns)
n_rows = int(top_n_features / 5) if top_n_features % 5 == 0 else int(top_n_features / 5) + 1
n_cols = 5
fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, n_rows*4))
axs = axs.flatten()


# plot each feature in a separate subplot
for i, feature in enumerate(top_feature_data.columns):
    x = top_feature_data[feature].values.ravel()  # ensure 1D
    y = target_data.values.ravel()            # ensure 1D
    axs[i].scatter(x, y, alpha=0.5)
    # add a dash linear regression line with a pearson correlation coefficient
    # compute the pearson correlation coefficient
    corr, _ = pearsonr(x, y)
    # add a linear regression line
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    y_pred = model.predict(x.reshape(-1, 1))
    axs[i].plot(x, y_pred, color='red', linestyle='--', linewidth=2)
    axs[i].set_title(f'{feature} (r={corr:.2f})')

    axs[i].set_xlabel('Feature value')
    axs[i].set_ylabel('Target value')
    axs[i].grid()
    # set the x and y limits to be the same for all subplots
    max_feature = top_feature_data[feature].values.max()   
    max_target = target_data.values.max()  
    axs[i].set_xlim([0,max_feature])
    axs[i].set_ylim([0,max_target])
    # compute the correlation between the feature and target data
plt.tight_layout()
plt.show()

In [ ]:
# plot the target data only with the x axis being 0 to 100 
plt.figure(figsize=(20, 10))
plt.hist(target_data, bins=100, alpha=0.5, color='blue')
plt.show()

### Suboptimal Model Creation

#### If modifying at the model architectural level

Mostly done to modify the `ModelSpec` object, where the updated spec information is transferred to a new `ModelBuilder` object, which then need to transpile to Antimony/SBML for a Solver instance. 

NewSpec --> NewBuilder --> Update parameters to original builder --> Transpile to Antimony/SBML --> Solver instance

#### If only changes to states and parameters are needed

Simply duplicate the `ModelBuilder` object and update the states and parameters.
```


#### Vary different specifications

In [ ]:
parameter_list = G0_d.get_parameters()
# make all parameters to zero
zero_parameter_list = {k: 0 for k in parameter_list.keys() if 'Km' not in k}
# make 1000 copies of the zero parameter list
zero_parameter_sets = [deepcopy(zero_parameter_list) for _ in range(1000)]


#### Vary by removing specific parameters

In [ ]:
# sort by mean rmse
ranked_regs = ranked_regs.sort_values(by='mean_rmse', ascending=False)
ranked_regs

In [ ]:
all_link_removal_parameters = []
parameters_to_zero = set()
# iterate ranked_regs from the top and remove the parameters that are in the zero_parameter_sets
for index, row in ranked_regs.iterrows():
    link_removal_parameter_sets = []
    parameters = row['parameters']
    for param in parameters:
        # if the parameter comtains 'Km' then skip it
        if 'Km' in param:
            continue
        parameters_to_zero.add(param)
    print(f'length of parameters to zero: {len(parameters_to_zero)}')
    for all_params in parameter_sets:
        if parameters is not None:
            new_params = all_params.copy()
            for zero_param in parameters_to_zero:
                if zero_param in all_params:
                    new_params[zero_param] = 0
            link_removal_parameter_sets.append(new_params)
    all_link_removal_parameters.append(link_removal_parameter_sets)
    
print(f'Number of link removal parameter sets: {len(all_link_removal_parameters)}')
            

### Simulation / Extract dynamic feature data

In [ ]:
# Minor distortion of the parameters to create a new set of parameters

rng = np.random.default_rng(o_random_seed)
modified_parameter_sets = []
for params in parameter_sets:
    new_params = {}
    for key, value in params.items(): 
        new_params[key] = value * rng.uniform(0.25, 4) # distortion range, expectation is that the larger the worse the model performance 
    modified_parameter_sets.append(new_params)

In [ ]:
from models.SyntheticGen import generate_model_timecourse_data_diff_build

# generate the timecourse data for the new model
time_course_data = generate_model_timecourse_data_diff_build(model_drug_spec, 
                                                  solver, 
                                                  feature_data, 
                                                  modified_parameter_sets,
                                                  {'start': 0, 'end': 1000, 'points': 100}, 
                                                  capture_species='all', n_cores=1, verbose=True)


In [ ]:
true_time_course_data = generate_model_timecourse_data_diff_build(model_drug_spec, 
                                                            solver,
                                                            feature_data, 
                                                            parameter_sets, 
                                                            {'start': 0, 'end': 1000, 'points': 100}, 
                                                            capture_species='all', n_cores=1, verbose=True)

In [ ]:
# link removal time course data (sets)

link_removal_time_course_datasets = []
for link_removal_params in all_link_removal_parameters:
    link_removal_time_course_data = generate_model_timecourse_data_diff_build(model_drug_spec, 
                                                                    solver, 
                                                                    feature_data, 
                                                                    link_removal_params, 
                                                                    {'start': 0, 'end': 1000, 'points': 100}, 
                                                                    capture_species='all', n_cores=1, verbose=True)
    link_removal_time_course_datasets.append(link_removal_time_course_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get time axis (assuming same length for both datasets)
n_steps = len(time_course_data['Cp'][0])
real_time = np.linspace(0, 1000, n_steps)

plt.figure(figsize=(10, 5))

# Plot true time course (blue)
for series in true_time_course_data['Cp']:
    plt.plot(real_time, series, alpha=0.3, color='blue', label='True Cp' if 'True Cp' not in plt.gca().get_legend_handles_labels()[1] else "")
    
# Plot simulated time course (red)
for series in time_course_data['Cp']:
    plt.plot(real_time, series, alpha=0.3, color='red', label='Simulated Cp' if 'Simulated Cp' not in plt.gca().get_legend_handles_labels()[1] else "")

# Labels and formatting
plt.title("Comparison of Simulated, True, and Original Cp Time Series")
plt.xlabel("Time")
plt.ylabel("Cp Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Set up time axis ===
n_steps = len(time_course_data['Cp'][0])  # assumes all rows have equal length
real_time = np.linspace(0, 1000, n_steps)

# === Helper function to plot mean ± std ===
def plot_mean_std(df, color, label, time_axis):
    # Stack rows of lists into a 2D array: shape (n_replicates, time_steps)
    data = np.vstack(df['Cp'].values)
    mean = data.mean(axis=0)
    std = data.std(axis=0)

    # Plot mean line
    plt.plot(time_axis, mean, color=color, label=label, linewidth=2)
    # Plot shaded std band
    plt.fill_between(time_axis, mean - std, mean + std, color=color, alpha=0.2)

# === Begin Plot ===
plt.figure(figsize=(10, 5))

# Plot mean ± std for true data
plot_mean_std(true_time_course_data, color='blue', label='True Cp', time_axis=real_time)

# Plot mean ± std for simulated data
# plot_mean_std(time_course_data, color='red', label='Simulated Cp', time_axis=real_time)

# Plot mean ± std for link removal datasets
for i, link_removal_data in enumerate(link_removal_time_course_datasets):
    plot_mean_std(link_removal_data, color=f'C{i+2}', label=f'Link Removal Set {i+1}', time_axis=real_time)

# === Plot Settings ===
plt.title("Comparison of Simulated, True, and Original Cp Time Series (Mean ± Std)")
plt.xlabel("Time")
plt.ylabel("Cp Value")
# plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Creating final datasets for ML Training

In [ ]:
from models.Utils import last_time_point_method, dynamic_features_method

all_species = model_drug_spec.A_species + model_drug_spec.B_species + model_drug_spec.C_species
all_phos_species = [s+'p' for s in all_species]
# apply the data engineering method to the feature data
last_time_data = last_time_point_method(time_course_data, all_phos_species)

In [ ]:
dynamic_data = dynamic_features_method(time_course_data, all_phos_species, n_cores=8)

In [ ]:
combined_lp_data = pd.concat([feature_data, last_time_data], axis=1)
combined_dyn_data = pd.concat([feature_data, dynamic_data], axis=1)


In [ ]:
feature_data_list = [feature_data, last_time_data, dynamic_data, combined_lp_data, combined_dyn_data]
feature_data_names = ['feature_data', 'last_time_data', 'dynamic_data', 'combined_lp_data', 'combined_dyn_data']

In [ ]:
last_time_data_link_rm = []
dynamic_data_link_rm = []
for zeroed_params in link_removal_time_course_datasets:
    last_time_data_rm = last_time_point_method(zeroed_params, all_phos_species)
    dynamic_data_rm = dynamic_features_method(zeroed_params, all_phos_species, n_cores=8)
    last_time_data_link_rm.append(last_time_data_rm)
    dynamic_data_link_rm.append(dynamic_data_rm)
    
# combine the link removal data with the feature data
link_removal_last_time_combined = []
link_removal_dynamic_combined = []
for i in range(len(last_time_data_link_rm)):
    link_removal_last_time_combined.append(pd.concat([feature_data, last_time_data_link_rm[i]], axis=1))
    link_removal_dynamic_combined.append(pd.concat([feature_data, dynamic_data_link_rm[i]], axis=1))
    
# create labels for the link removal data
link_removal_last_time_labels = [f'link_removal_{i+1}_last_time' for i in range(len(last_time_data_link_rm))]
link_removal_dynamic_labels = [f'link_removal_{i+1}_dynamic' for i in range(len(dynamic_data_link_rm))]
# create labels for the combined data
combined_lp_labels = [f'link_removal_{i+1}_combined_lp' for i in range(len(link_removal_last_time_combined))]
combined_dyn_labels = [f'link_removal_{i+1}_combined_dyn' for i in range(len(link_removal_dynamic_combined))]

# add into feature_data_list and feature_data_names
feature_data_list.extend(last_time_data_link_rm)
feature_data_list.extend(dynamic_data_link_rm)
feature_data_list.extend(link_removal_last_time_combined)
feature_data_list.extend(link_removal_dynamic_combined)
feature_data_names.extend(link_removal_last_time_labels)
feature_data_names.extend(link_removal_dynamic_labels)
feature_data_names.extend(combined_lp_labels)
feature_data_names.extend(combined_dyn_labels)

### Machine Learning Training

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def build_pipeline(model, scale=False):
    steps = [('imputer', SimpleImputer(strategy='mean'))]
    if scale:
        steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    return Pipeline(steps)



def evaluate_model(model, model_name, feature_data, feature_data_name, target_data, test_size=0.2, random_state=4):
    # Align rows between X and y
    common_idx = feature_data.index.intersection(target_data.index)
    X = feature_data.loc[common_idx]
    y = target_data.loc[common_idx]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return {
        'Model': model_name, 
        'Feature Data': feature_data_name,
        'Mean Squared Error': mean_squared_error(y_test, y_pred),
        'R2 Score': r2_score(y_test, y_pred),
        'Pearson Correlation': pearsonr(y_test, y_pred)[0],
        'Pearson P-Value': pearsonr(y_test, y_pred)[1]
    }

all_models = [
    build_pipeline(LinearRegression()),
    build_pipeline(RandomForestRegressor(n_estimators=100, random_state=o_random_seed)),
    build_pipeline(GradientBoostingRegressor(n_estimators=100, random_state=o_random_seed)),
    build_pipeline(SVR(max_iter=10000), scale=True),
    build_pipeline(MLPRegressor(hidden_layer_sizes=(20,), max_iter=10000, random_state=o_random_seed), scale=True)
]

all_models_desc = ['Linear Regression', 'Random Forest', 'Gradient Boosting', 'Support Vector Machine', 'Neural Network']
zipped_model_data = list(zip(all_models, all_models_desc))
all_features = feature_data_list
all_features_desc = feature_data_names
zipped_feature_data = list(zip(all_features, all_features_desc))

# random states are rand ints between 0 and 10000, for n values 
np.random.seed(o_random_seed)
n_random = 10
all_random_states = np.random.randint(0, 10000, n_random)

parallelise = True 
from tqdm import tqdm
# tqdm is a progress bar library, use it to show the progress of the model evaluation
metric_data = []
if not parallelise:          
    for (feature_data, feature_data_name) in tqdm(zipped_feature_data):
        # print('Feature Data:', feature_data_name)
        # print('Feature Data Shape:', feature_data.shape)
        for (model, model_name) in zipped_model_data:
            # print('Model:', model_name)
            for rand in all_random_states:
                metrics = evaluate_model(model, model_name, feature_data, feature_data_name, target_data['Cp'], random_state=rand)
                metric_data.append(metrics)
                
else:        
    # parallelise the model evaluation process using joblib
    from joblib import Parallel, delayed

    metric_data = Parallel(n_jobs=-1)(delayed(evaluate_model)(model, model_name, feature_data, feature_data_name, target_data['Cp'], random_state=rand) 
                                    for (feature_data, feature_data_name) in zipped_feature_data
                                    for (model, model_name) in zipped_model_data
                                    for rand in all_random_states)

# make a dataframe of the metric data
metric_df = pd.DataFrame(metric_data)
metric_df

In [ ]:
if save_figures: 
    metric_df.to_pickle(experiment_folder+'metric_df.pkl')
    print('Metric data saved to:', experiment_folder+'metric_df.pkl')   


In [ ]:
# load the metric data
import pandas as pd 
metric_df = pd.read_pickle(experiment_folder+'metric_df.pkl')
print('Metric data loaded from:', experiment_folder+'metric_df.pkl')

In [ ]:
import pandas as pd
import re

# Apply regex to extract parts from 'Feature Data'
def extract_link_removal_parts(feature_str):
    match = re.match(r'(link_removal_(\d+)_([\w]+))', feature_str)
    if match:
        full_name = match.group(1)
        removed_links = int(match.group(2))
        data_type = match.group(3)
        return pd.Series([full_name, removed_links, data_type])
    else:
        return pd.Series([feature_str, None, None])  # fallback for non-matching entries

# Apply to metric_df
metric_df[['Full name', 'Removed links', 'Type']] = metric_df['Feature Data'].apply(extract_link_removal_parts)



In [ ]:
metric_df

In [ ]:
# plot the pearson correlation for each feature data combination

import matplotlib.pyplot as plt
import seaborn as sns
# Set up the figure
plt.figure(figsize=(120, 6))
# Create a boxplot for Pearson Correlation
sns.boxplot(data=metric_df, x='Feature Data', y='Pearson Correlation', hue='Feature Data')
plt.xticks(rotation=45, ha='right')
plt.title('Pearson Correlation by Feature Data and Model')
plt.xlabel('Feature Data')
plt.ylabel('Pearson Correlation')
plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# FILL missing values of Pearson Correlation with 0 
metric_df['Pearson Correlation'] = metric_df['Pearson Correlation'].fillna(0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("talk", rc={"font": "Arial"})
sns.set_style('whitegrid')

# Group and average
grouped = (
    metric_df
    .dropna(subset=['Removed links', 'Pearson Correlation', 'Type'])
    .groupby(['Removed links', 'Type'])
    .agg(mean_corr=('Pearson Correlation', 'mean'))
    .reset_index()
)

plt.figure(figsize=(10, 6))
sns.lineplot(data=grouped, x='Removed links', y='mean_corr', hue='Type', marker='o')
plt.title('Mean Pearson Correlation vs Removed Links')
plt.xlabel('Number of Links Removed')
plt.ylabel('Mean Pearson Correlation')
plt.tight_layout()
plt.show()
